<a href="https://colab.research.google.com/github/Nirupama06/Sign_to_Text/blob/main/SigntoText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                            validation_split = 0.1,rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [ ]:
train_ds  = datagen.flow_from_directory(
       'Documents/asl_alphabet_train/asl_alphabet_train',
        target_size=(100, 100),
        batch_size=1024,
        subset='training',
        class_mode='categorical')

In [ ]:
val_ds  = datagen.flow_from_directory(
           'Documents/asl_alphabet_train/asl_alphabet_train',
        target_size=(100, 100),
        batch_size=1024,
        subset='validation',
        class_mode='categorical')

In [ ]:
img, lab = next(iter(train_ds ))
img.shape, lab.shape
class_indices = train_ds.class_indices
class_indices
lab

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Dense , Dropout
import os
sz=128

In [ ]:
model = Sequential()


model.add(Convolution2D(16, (3, 3), input_shape=(100, 100, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())

model.add(Flatten())


model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(units=500, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=29, activation='softmax')) # softmax for more than 2


model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2



model.summary()
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
batch_size=100
epochs=30

history = model.fit(train_ds,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_ds,
                    callbacks = [early_stop])

In [ ]:
test_image = tf.io.gfile.listdir(r'Documents/asl_alphabet_test/asl_alphabet_test')
images = []
for f in test_image:
    img = tf.keras.preprocessing.image.load_img(r'Documents/asl_alphabet_test/asl_alphabet_test/'+ f, color_mode = "rgb", target_size = (100,100))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img=img/255
    images.append(img)
test=np.array(images)
pred = model.predict(test)
pred
model.save('Documents/model_predict.h5')

In [ ]:
import numpy as np
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
bg = None
global camera

def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [ ]:
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(),
                                 cv2.RETR_EXTERNAL,
                                 cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)


In [ ]:
import imutils
def main():
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0
    image_num = 0
    val = 0

    start_recording = False

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()
        if (grabbed == True):

            # resize the frame
            frame = imutils.resize(frame, width=700)

            # flip the frame so that it is not the mirror view
            frame = cv2.flip(frame, 1)

            # clone the frame
            clone = frame.copy()

            # get the height and width of the frame
            (height, width) = frame.shape[:2]

            # get the ROI
            roi = frame[top:bottom, right:left]

            # convert the roi to grayscale and blur it
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            #gray = cv2.GaussianBlur(gray, (7, 7), 0)

            # to get the background, keep looking till a threshold is reached
            # so that our running average model gets calibrated
            if num_frames < 30:
                run_avg(gray, aWeight)
#                 print(num_frames)
            else:
                # segment the hand region
                hand = segment(gray)

                # check whether hand region is segmented
                if hand is not None:
                    # if yes, unpack the thresholded image and
                    # segmented region
                    (thresholded, segmented) = hand

                    # draw the segmented region and display the frame
#                     cv2.drawContours(
#                         clone, [segmented + (right, top)], -1, (0, 0, 255))

                    if start_recording and num_frames%5==0:
                        frame_topredict = cv2.resize(frame, (32, 32)).reshape(1, 32, 32, 3)
                        preds = model.predict(frame_topredict)
                        val = np.argmax(preds[0])
                        print(val)
        
                    cv2.putText(clone, class_names[val], (right, bottom+15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                        
                        
                        # Mention the directory in which you wanna store the images followed by the image name
#                         cv2.imwrite("Downloads/Dataset/FistTest/fist_" +
#                                     str(image_num) + '.png', thresholded)
#                         image_num += 1
#                         print(image_num)
#                     cv2.imshow("Thesholded", thresholded)

            # draw the segmented hand
            cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

            # increment the number of frames
            num_frames += 1

            # display the frame with segmented hand
            cv2.imshow("Video Feed", clone)

            # observe the keypress by the user
            keypress = cv2.waitKey(1) & 0xFF

            # if the user pressed "q", then stop looping
            if keypress == ord("q"):
                break

            if keypress == ord("s"):
                start_recording = True
                num_frames = 0

        else:
            print("[Warning!] Error input, Please check your(camera Or video)")
            break
            
    camera.release()


In [ ]:

main()

# free up memory

#videoCaptureObject = cv2.VideoCapture(0)
#result = True
#while(result):
  #  ret,frame = videoCaptureObject.read()
  #  cv2.imwrite("NewPicture.jpg",frame)
   # result = False
#videoCaptureObject.release()
#cv2.destroyAllWindows()
# camera.release()
cv2.destroyAllWindows()




import cv2

videoCaptureObject = cv2.VideoCapture(0)
result = True
while(result):
    ret,frame = videoCaptureObject.read()
    cv2.imwrite("NewPicture.jpg",frame)
    result = False
videoCaptureObject.release()
cv2.destroyAllWindows()




get_ipython().run_line_magic('pwd', '')